In [ ]:
import pandas as pd
import numpy as np

men_pmsi = pd.read_csv("../../../data_test/diamant/MEN_PMSI_ANNUEL_2022_10_25.CSV", sep=";")

men_pmsi.head()


In [ ]:
had = men_pmsi[["Finess EJ", "Finess", "Année", "Nombre total de séjours HAD"]]
had.head()

In [ ]:
had["Nombre total de séjours HAD"].info()

In [ ]:
# Pourcentage des valeurs N/A dans la colonne Nombre total de sejours HAD (chaque annee, EJ, ET)

na_had = had["Nombre total de séjours HAD"].isna().sum()

total_had = len(had)

pourcentage_na = na_had/total_had * 100

print("Percentage of Na in dataset is: ", pourcentage_na)

In [ ]:
had_grouped = had.groupby(["Finess EJ", "Année"])["Nombre total de séjours HAD"].sum()
had_grouped.head()


In [ ]:
# Total nombre de 0

zero_had = (had["Nombre total de séjours HAD"] == 0).sum()

print("Total number rows with value 0 is: ", zero_had)

In [ ]:
negative_had = (had["Nombre total de séjours HAD"] < 0).sum()

print("Total number rows with value negative is: ", negative_had)